# 2. Vector Search

In [1]:
%%writefile requirements.txt
python-dotenv>=1.0
kaggle
kagglehub
numpy
pandas
matplotlib
seaborn

# Prevent Chroma import crash
pydantic<2.12
pydantic-settings<2.12

torch
transformers

# Keep your current working LangChain/Chroma combo stable
chromadb==1.5.0
langchain==1.2.10
langchain-chroma==1.1.0
langchain-community==0.4.1
langchain-core==1.2.12
langchain-openai==1.1.9
langchain-text-splitters==1.1.0

Writing requirements.txt


In [2]:
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 52.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.1/500.1 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [4]:
# Setup Environment
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('../../.env')
# Access the OpenAI API key from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
huggingfacehub_api_key = os.getenv('HUGGINGFACEHUB_API_TOKEN')
print("Environment variables loaded successfully.")
print(f"OpenAI API Key: {'Loaded' if openai_api_key else 'Not Loaded'}")
print(f"Hugging Face Hub API Key: {'Loaded' if huggingfacehub_api_key else 'Not Loaded'}")

Environment variables loaded successfully.
OpenAI API Key: Not Loaded
Hugging Face Hub API Key: Not Loaded


In [ ]:
import pandas as pd
books = pd.read_csv('../data/processed/moviez.csv')

In [ ]:
moviez.head()